## Connexion postgres

In [1]:
import os
from dotenv import load_dotenv
import psycopg2

# Charger les variables d'environnement depuis le fichier .env
load_dotenv(".env")

USER_PSQL = os.environ.get("POSTGRESQL_LOCAL_USER")
PASSWORD_PSQL = os.environ.get("POSTGRESQL_LOCAL_PASSWORD")

# Se connecter à la base de données PostgreSQL
conn = psycopg2.connect(user=USER_PSQL, password=PASSWORD_PSQL, host="localhost", port="5432", database="app_auth")
cursor = conn.cursor()

## Création de la table username_history_from_app

In [2]:
create_table_script = """
CREATE TABLE IF NOT EXISTS username_history_from_app (
    history_id SERIAL PRIMARY KEY,
    user_id INT REFERENCES "user" (user_id),
    username_new VARCHAR(255)
);
"""
cursor.execute(create_table_script)
conn.commit()

## Comparaison des Temps d'Exécution

In [3]:
import time

user_id = 2047  

def update_username_history(new_username):
    start = time.time()
    cursor = conn.cursor()
    
    insert_query = f"INSERT INTO username_history (user_id, username_new) VALUES (%s, %s)"
    cursor.execute(insert_query, (user_id, new_username))

    conn.commit()
    end = time.time()

    total = end - start
    return total

def update_user_with_trigger(new_username):
    start = time.time()
    cursor = conn.cursor()

    update_query = f"UPDATE username_history_from_app SET username_new = %s WHERE user_id = %s"
    cursor.execute(update_query, (new_username, user_id))

    conn.commit()
    end = time.time()
    total = end - start

    return total


def main():
    
    for num_edits in [100, 1000, 10000]:

        time_query_with_trigger = sum(update_user_with_trigger(f"username{i}") for i in range(num_edits))

        time_query_without_trigger = sum(update_username_history(f"username{i}") for i in range(num_edits))


        print(f"pour {num_edits} éditions:")
        print(f"Avec trigger: {time_query_with_trigger} s")
        print(f"Sans trigger: {time_query_without_trigger} s")

main()

pour 100 éditions:
Avec trigger: 0.07626104354858398 s
Sans trigger: 0.10744118690490723 s
pour 1000 éditions:
Avec trigger: 1.0237491130828857 s
Sans trigger: 1.355050802230835 s
pour 10000 éditions:
Avec trigger: 9.059243202209473 s
Sans trigger: 11.081467390060425 s
